Hi
I wanted to see if there was statistically significant lowered stock market performance on the first day of the year following a year of gains in the stock market as it might be that for tax reasons investors looking to sell their stocks may wait for the first day of the new tax year. I got the data off https://www.investing.com/ for the S&P 500 and combined the csvs and reduced it to the data and Change % columns in excel before importing it here.

In [165]:
import pandas as pd
from scipy.stats import ttest_ind

In [166]:
# Reading csv into dataframe
daily_df = pd.read_csv('dailystockmarketdata.csv')

# converting 'Date' column to datetime
daily_df['Date'] = pd.to_datetime(daily_df['Date'])

# Group by year and get the row with the minimum date in each year
filtered_df = daily_df.groupby(daily_df['Date'].dt.year)['Date'].idxmin()

# Filter the original DataFrame based on the indices obtained
result_df = daily_df.loc[filtered_df]

# needed to get rid of the first entry as the data starts in december of 1979
first_day_data = result_df.drop(result_df.index[0])


Apparently investing.com has more data available for monthly versus daily which is why I'm importing this other
data I downloaded and then cleaned a bit in excel again. I'm trying to figure out which years the s%p 500 saw an increase

In [167]:
# Reading csv into dataframe
monthly_df = pd.read_csv('monthlystockmarketdata.csv')

# Convert 'Date' column to datetime format
monthly_df['Date'] = pd.to_datetime(df['Date'])  # Convert 

# Create a new column 'Year' with the extracted year
monthly_df['Year'] = monthly_df['Date'].dt.year

# Calculate the monthly change factors (1 + change)
monthly_df['Monthly Change Factor'] = 1 + monthly_df['Change %'] / 100

# Group by year and calculate the annual change
df_grouped = monthly_df.groupby('Year')['Monthly Change Factor'].prod() - 1
df_grouped = df_grouped.reset_index()
df_grouped.columns = ['Year', 'Annual Change']

# Filter rows where the stock market grew over the year
df_grouped_filtered = df_grouped[df_grouped['Annual Change'] > 0]


In [168]:

# Convert 'Date' column in first_day_data to string
first_day_data['Date'] = first_day_data['Date'].astype(str)

# Extract unique year strings from 'Year' column in df_grouped_filtered
unique_year_strings = df_grouped_filtered['Year'].astype(str).unique()

# Filter data based on the criteria
filtered_first_day_data = first_day_data[first_day_data['Date'].str.startswith(tuple(unique_year_strings))]


In [169]:

# Merge the DataFrames on the 'Date' column with indicator set to True
merged_df = pd.merge(daily_data, filtered_first_day_data, how='outer', indicator=True)

# Select rows that are only in daily_data (not in filtered_first_day_data)
result_df = merged_df[merged_df['_merge'] == 'left_only']

# Drop the indicator column if needed
other_days_data = result_df.drop('_merge', axis=1)

In [178]:
from scipy.stats import ttest_ind


# Filter data for January 2nd and other dates
filtered_first_day_data_returns = filtered_first_day_data['Change %']
other_days_data_returns = other_days_data['Change %']

# Performing t-test
t_stat, p_value = ttest_ind(filtered_first_day_data_returns, other_days_data_returns, equal_var=False)

# alpha = 0.05 just to be conservative
alpha = 0.05

# Print the results
print("T-statistic:", t_stat)
print("P-value:", p_value)

# Check for statistical significance
if p_value < alpha:
    print("The average performance on January 2nd is statistically significant.")
else:
    print("There is no statistically significant difference in average performance of the s&p 500 on the first day of the year after a year of growth compared to a normal day.")


T-statistic: 0.9898950691463758
P-value: 0.3293998408928476
There is no statistically significant difference in average performance of the s&p 500 on the first day of the year after a year of growth compared to a normal day.
